In [121]:
#Data Structures
import pandas as pd
import numpy as np
import re
import os

### For installing missingno library, type this command in terminal
#pip install missingno

import missingno as msno

#Sklearn
import joblib
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score,confusion_matrix, precision_score, recall_score, f1_score

#Plotting
import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns

#Others
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

### Preparing Test data

In [122]:
df_test = pd.read_csv('test.csv')

In [123]:
df_test.head()

,id,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,arpu_6,arpu_7,...,sachet_3g_6,sachet_3g_7,sachet_3g_8,fb_user_6,fb_user_7,fb_user_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g
0,69999,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,91.882,65.330,...,0,0,0,NaN,NaN,NaN,1692,0.00,0.00,0.00
1,70000,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,414.168,515.568,...,0,0,0,NaN,NaN,NaN,2533,0.00,0.00,0.00
2,70001,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,329.844,434.884,...,0,0,0,NaN,NaN,NaN,277,525.61,758.41,241.84
3,70002,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,43.550,171.390,...,0,0,0,NaN,NaN,NaN,1244,0.00,0.00,0.00
4,70003,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,306.854,406.289,...,0,0,0,NaN,NaN,NaN,462,0.00,0.00,0.00


In [124]:
#Let's load the train data used in training to select the top features
X_train_rfe = pd.read_csv('X_train_rfe.csv')
X_train_rfe.head()

,arpu_6,arpu_7,arpu_8,onnet_mou_6,onnet_mou_7,onnet_mou_8,offnet_mou_6,offnet_mou_7,offnet_mou_8,roam_ic_mou_7,...,total_rech_amt_8,max_rech_amt_6,max_rech_amt_7,last_day_rch_amt_8,vol_2g_mb_7,vol_2g_mb_8,vol_3g_mb_8,monthly_2g_7,aon,aug_vbc_3g
0,-0.302124,0.535981,0.658669,0.658549,1.465721,1.561153,-0.893141,-0.036487,0.514227,-0.385211,...,0.628332,0.474316,0.487723,1.037975,-0.476943,-0.457622,-0.419241,-0.286104,-0.055152,-0.423562
1,0.470038,-0.732782,-2.906627,0.620661,0.161087,-0.681293,0.817416,-0.635504,-1.036710,-0.385211,...,-2.288544,0.569031,-2.422674,-1.304482,-0.476943,-0.478413,-0.419241,-0.286104,-0.785938,-0.423562
2,0.192098,0.240227,0.787816,1.059716,1.335814,1.795646,0.256792,0.717342,0.108677,-0.385211,...,0.710113,0.474316,0.487723,-1.304482,-0.476943,-0.478413,-0.419241,-0.286104,0.154209,-0.423562
3,-0.033455,0.069898,0.124634,0.625248,0.522717,0.607030,-1.311764,-0.391315,-0.404600,-0.385211,...,0.232208,-0.329589,-0.300537,0.345496,-0.476943,-0.478413,-0.419241,-0.286104,-0.685677,-0.423562
4,0.240241,0.171304,0.253336,-0.609620,-0.274470,-0.506961,0.611961,0.457681,0.476545,-0.385211,...,0.363316,0.988547,0.991947,1.350413,-0.476943,-0.473138,2.114952,-0.286104,1.156711,0.501534


In [125]:
#Let's load the X_train without scaling as well
X_train = pd.read_csv('X_train.csv')
X_train.head()

,arpu_6,arpu_7,arpu_8,onnet_mou_6,onnet_mou_7,onnet_mou_8,offnet_mou_6,offnet_mou_7,offnet_mou_8,roam_ic_mou_6,...,monthly_3g_6,monthly_3g_7,monthly_3g_8,sachet_3g_6,sachet_3g_7,sachet_3g_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g
0,4.539137,5.772285,5.973759,4.590665,6.100409,6.255328,2.814210,4.168369,5.054269,0.936093,...,0.000000,0.0,0.0,0.0,0.0,0.0,6.741701,0.000000,0.0,0.0
1,5.702532,3.840742,0.000000,4.518958,3.602232,1.943049,5.612361,3.173041,2.421257,0.000000,...,0.693147,0.0,0.0,0.0,0.0,0.0,6.137727,0.000000,0.0,0.0
2,5.283767,5.322034,6.190147,5.349913,5.851656,6.706263,4.695285,5.420933,4.365770,2.364620,...,0.693147,0.0,0.0,0.0,0.0,0.0,6.914731,0.000000,0.0,0.0
3,4.943933,5.062728,5.078967,4.527641,4.294697,4.420526,2.129421,3.578786,3.494384,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,6.220590,0.000000,0.0,0.0
4,5.356303,5.217107,5.294611,2.190536,2.768204,2.278292,5.276276,4.989480,4.990297,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,7.743270,1.848455,0.0,0.0


In [126]:
#now let's take the rfe selected features for scalling
X_train_rfe = X_train[X_train_rfe.columns]

In [127]:
#Let's separate the 50 features first
df_test = df_test[X_train_rfe.columns]

In [128]:
df_test.head()

,arpu_6,arpu_7,arpu_8,onnet_mou_6,onnet_mou_7,onnet_mou_8,offnet_mou_6,offnet_mou_7,offnet_mou_8,roam_ic_mou_7,...,total_rech_amt_8,max_rech_amt_6,max_rech_amt_7,last_day_rch_amt_8,vol_2g_mb_7,vol_2g_mb_8,vol_3g_mb_8,monthly_2g_7,aon,aug_vbc_3g
0,91.882,65.330,64.445,31.78,20.23,23.11,60.16,32.16,34.83,0.00,...,60,50,30,0,0.0,0.0,0.0,0,1692,0.00
1,414.168,515.568,360.868,75.51,41.21,19.84,474.34,621.84,394.94,0.00,...,500,250,250,0,0.0,0.0,0.0,0,2533,0.00
2,329.844,434.884,746.239,7.54,7.86,8.40,16.98,45.81,45.04,103.38,...,1000,300,500,0,0.0,0.0,0.0,0,277,525.61
3,43.550,171.390,24.400,5.31,2.16,0.00,40.04,205.01,24.01,0.00,...,0,110,150,0,0.0,0.0,0.0,0,1244,0.00
4,306.854,406.289,413.329,450.93,609.03,700.68,60.94,23.84,74.16,0.00,...,546,90,130,130,0.0,0.0,0.0,0,462,0.00


#### Comparing the nature of test data with train

In [129]:
#Let's look at overall view of distribution in data
skewness = df_test.skew()
skewness = pd.DataFrame(skewness).reset_index()
skewness = skewness.rename(columns = {'index':'columns',0:'sk_value'})
skewness = skewness.sort_values(by = 'sk_value', key = abs, ascending = False)
skewness.head()
#Clearly all the skewness values in the dataset is positive indicating 
#The overall data is rightly skewed towards high values
#So the behaviour is similar to train data

,columns,sk_value
9,roam_ic_mou_7,116.101278
22,og_others_8,115.258328
30,std_ic_t2f_mou_8,35.974763
20,isd_og_mou_8,34.667280
28,std_ic_t2t_mou_8,31.517918


In [130]:
## Let's look at the percentage of missing values
(df_test.isnull().sum()/df_test.shape[0])*100

arpu_6                0.000000
arpu_7                0.000000
arpu_8                0.000000
onnet_mou_6           3.896667
onnet_mou_7           3.906667
onnet_mou_8           5.583333
offnet_mou_6          3.896667
offnet_mou_7          3.906667
offnet_mou_8          5.583333
roam_ic_mou_7         3.906667
roam_og_mou_7         3.906667
roam_og_mou_8         5.583333
loc_og_mou_7          3.906667
loc_og_mou_8          5.583333
std_og_t2t_mou_6      3.896667
std_og_t2t_mou_8      5.583333
std_og_t2m_mou_6      3.896667
std_og_t2f_mou_6      3.896667
std_og_mou_6          3.896667
std_og_mou_8          5.583333
isd_og_mou_8          5.583333
spl_og_mou_8          5.583333
og_others_8           5.583333
total_og_mou_6        0.000000
total_og_mou_7        0.000000
total_og_mou_8        0.000000
loc_ic_t2m_mou_8      5.583333
loc_ic_mou_8          5.583333
std_ic_t2t_mou_8      5.583333
std_ic_t2f_mou_6      3.896667
std_ic_t2f_mou_8      5.583333
std_ic_mou_6          3.896667
std_ic_m

### Outlier Handling

In [131]:
#Let's first replace all the negative values in arpu with 0
df_test.loc[df_test['arpu_6'] < 0, 'arpu_6'] = 0
df_test.loc[df_test['arpu_7'] < 0, 'arpu_7'] = 0
df_test.loc[df_test['arpu_8'] < 0, 'arpu_8'] = 0  

In [132]:
def outliers_z(data,k_right = 3):
    upper_limit = data.mean() + k_right*data.std()
    print('number of outliers', len(data[data>upper_limit]))
    data[data>upper_limit] = upper_limit
    return data

In [133]:
df_test = df_test.apply(outliers_z, axis=0)

number of outliers 475
number of outliers 501
number of outliers 506
number of outliers 626
number of outliers 620
number of outliers 592
number of outliers 558
number of outliers 528
number of outliers 515
number of outliers 118
number of outliers 352
number of outliers 354
number of outliers 527
number of outliers 525
number of outliers 631
number of outliers 610
number of outliers 532
number of outliers 314
number of outliers 636
number of outliers 593
number of outliers 108
number of outliers 298
number of outliers 12
number of outliers 634
number of outliers 625
number of outliers 621
number of outliers 456
number of outliers 468
number of outliers 205
number of outliers 228
number of outliers 213
number of outliers 333
number of outliers 319
number of outliers 541
number of outliers 544
number of outliers 880
number of outliers 258
number of outliers 609
number of outliers 443
number of outliers 459
number of outliers 489
number of outliers 534
number of outliers 427
number of ou

### Let's Now deal with the missing vlues

In [134]:
#Let's first do imputation using KNN for nuemrical_df_z

import pandas as pd
from sklearn.impute import KNNImputer

# Create a KNN imputer object
imputer = KNNImputer(n_neighbors=5)  

# Impute missing values in the selected features
df_test = imputer.fit_transform(df_test)


In [135]:
df_test = pd.DataFrame(df_test, columns = X_train_rfe.columns)

In [136]:
#Verifying the null values again
(df_test.isnull().sum()/df_test.shape[0])*100

arpu_6                0.0
arpu_7                0.0
arpu_8                0.0
onnet_mou_6           0.0
onnet_mou_7           0.0
onnet_mou_8           0.0
offnet_mou_6          0.0
offnet_mou_7          0.0
offnet_mou_8          0.0
roam_ic_mou_7         0.0
roam_og_mou_7         0.0
roam_og_mou_8         0.0
loc_og_mou_7          0.0
loc_og_mou_8          0.0
std_og_t2t_mou_6      0.0
std_og_t2t_mou_8      0.0
std_og_t2m_mou_6      0.0
std_og_t2f_mou_6      0.0
std_og_mou_6          0.0
std_og_mou_8          0.0
isd_og_mou_8          0.0
spl_og_mou_8          0.0
og_others_8           0.0
total_og_mou_6        0.0
total_og_mou_7        0.0
total_og_mou_8        0.0
loc_ic_t2m_mou_8      0.0
loc_ic_mou_8          0.0
std_ic_t2t_mou_8      0.0
std_ic_t2f_mou_6      0.0
std_ic_t2f_mou_8      0.0
std_ic_mou_6          0.0
std_ic_mou_8          0.0
total_ic_mou_7        0.0
total_ic_mou_8        0.0
spl_ic_mou_8          0.0
isd_ic_mou_8          0.0
total_rech_num_8      0.0
total_rech_a

In [137]:
df_test.head()

,arpu_6,arpu_7,arpu_8,onnet_mou_6,onnet_mou_7,onnet_mou_8,offnet_mou_6,offnet_mou_7,offnet_mou_8,roam_ic_mou_7,...,total_rech_amt_8,max_rech_amt_6,max_rech_amt_7,last_day_rch_amt_8,vol_2g_mb_7,vol_2g_mb_8,vol_3g_mb_8,monthly_2g_7,aon,aug_vbc_3g
0,91.882,65.330,64.445,31.78,20.23,23.11,60.16,32.16,34.83,0.00,...,60.0,50.0,30.0,0.0,0.0,0.0,0.0,0.0,1692.0,0.00
1,414.168,515.568,360.868,75.51,41.21,19.84,474.34,621.84,394.94,0.00,...,500.0,250.0,250.0,0.0,0.0,0.0,0.0,0.0,2533.0,0.00
2,329.844,434.884,746.239,7.54,7.86,8.40,16.98,45.81,45.04,103.38,...,1000.0,300.0,500.0,0.0,0.0,0.0,0.0,0.0,277.0,525.61
3,43.550,171.390,24.400,5.31,2.16,0.00,40.04,205.01,24.01,0.00,...,0.0,110.0,150.0,0.0,0.0,0.0,0.0,0.0,1244.0,0.00
4,306.854,406.289,413.329,450.93,609.03,700.68,60.94,23.84,74.16,0.00,...,546.0,90.0,130.0,130.0,0.0,0.0,0.0,0.0,462.0,0.00


### Data Transformation

In [138]:
#Before applying log transformation let's first verify
#All the values in the dataset are positive

np.all(df_test >= 0)

True

In [139]:
#Let's tranform the data to log
X_log_test = np.log(df_test+1)
X_log_test.head()

,arpu_6,arpu_7,arpu_8,onnet_mou_6,onnet_mou_7,onnet_mou_8,offnet_mou_6,offnet_mou_7,offnet_mou_8,roam_ic_mou_7,...,total_rech_amt_8,max_rech_amt_6,max_rech_amt_7,last_day_rch_amt_8,vol_2g_mb_7,vol_2g_mb_8,vol_3g_mb_8,monthly_2g_7,aon,aug_vbc_3g
0,4.531330,4.194642,4.181210,3.489819,3.055415,3.182627,4.113493,3.501344,3.578786,0.000000,...,4.110874,3.931826,3.433987,0.000000,0.0,0.0,0.0,0.0,7.434257,0.00000
1,6.028683,6.247207,5.891280,4.337421,3.742657,3.036874,6.164030,6.434290,5.981263,0.000000,...,6.216606,5.525453,5.525453,0.000000,0.0,0.0,0.0,0.0,7.837554,0.00000
2,5.801647,6.077376,6.616385,2.144761,2.181547,2.240710,2.889260,3.846097,3.829511,4.648038,...,6.908755,5.707110,6.216606,0.000000,0.0,0.0,0.0,0.0,5.627621,6.26646
3,3.796612,5.149759,3.234749,1.842136,1.150572,0.000000,3.714547,5.327925,3.219276,0.000000,...,0.000000,4.709530,5.017280,0.000000,0.0,0.0,0.0,0.0,7.126891,0.00000
4,5.729626,6.009523,6.026660,6.113527,6.413508,6.553477,4.126166,3.212455,4.319619,0.000000,...,6.304449,4.510860,4.875197,4.875197,0.0,0.0,0.0,0.0,6.137727,0.00000


In [140]:
X_train_rfe.head()

,arpu_6,arpu_7,arpu_8,onnet_mou_6,onnet_mou_7,onnet_mou_8,offnet_mou_6,offnet_mou_7,offnet_mou_8,roam_ic_mou_7,...,total_rech_amt_8,max_rech_amt_6,max_rech_amt_7,last_day_rch_amt_8,vol_2g_mb_7,vol_2g_mb_8,vol_3g_mb_8,monthly_2g_7,aon,aug_vbc_3g
0,4.539137,5.772285,5.973759,4.590665,6.100409,6.255328,2.814210,4.168369,5.054269,0.0,...,6.077642,4.709530,4.709530,4.875197,0.0,0.039221,0.000000,0.0,6.741701,0.000000
1,5.702532,3.840742,0.000000,4.518958,3.602232,1.943049,5.612361,3.173041,2.421257,0.0,...,0.000000,4.859812,0.000000,0.000000,0.0,0.000000,0.000000,0.0,6.137727,0.000000
2,5.283767,5.322034,6.190147,5.349913,5.851656,6.706263,4.695285,5.420933,4.365770,0.0,...,6.248043,4.709530,4.709530,0.000000,0.0,0.000000,0.000000,0.0,6.914731,0.000000
3,4.943933,5.062728,5.078967,4.527641,4.294697,4.420526,2.129421,3.578786,3.494384,0.0,...,5.252273,3.433987,3.433987,3.433987,0.0,0.000000,0.000000,0.0,6.220590,0.000000
4,5.356303,5.217107,5.294611,2.190536,2.768204,2.278292,5.276276,4.989480,4.990297,0.0,...,5.525453,5.525453,5.525453,5.525453,0.0,0.009950,5.719164,0.0,7.743270,1.848455


In [141]:
#Let's apply Logarithmic transformation
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_rfe)

#Let'd transform on the 
X_test_scaled = scaler.transform(X_log_test)

In [142]:
X_train_scaled = pd.DataFrame(X_train_scaled, columns = X_train_rfe.columns)
X_train_scaled.head()

,arpu_6,arpu_7,arpu_8,onnet_mou_6,onnet_mou_7,onnet_mou_8,offnet_mou_6,offnet_mou_7,offnet_mou_8,roam_ic_mou_7,...,total_rech_amt_8,max_rech_amt_6,max_rech_amt_7,last_day_rch_amt_8,vol_2g_mb_7,vol_2g_mb_8,vol_3g_mb_8,monthly_2g_7,aon,aug_vbc_3g
0,-0.302124,0.535981,0.658669,0.658549,1.465721,1.561153,-0.893141,-0.036487,0.514227,-0.385211,...,0.628332,0.474316,0.487723,1.037975,-0.476943,-0.457622,-0.419241,-0.286104,-0.055152,-0.423562
1,0.470038,-0.732782,-2.906627,0.620661,0.161087,-0.681293,0.817416,-0.635504,-1.036710,-0.385211,...,-2.288544,0.569031,-2.422674,-1.304482,-0.476943,-0.478413,-0.419241,-0.286104,-0.785938,-0.423562
2,0.192098,0.240227,0.787816,1.059716,1.335814,1.795646,0.256792,0.717342,0.108677,-0.385211,...,0.710113,0.474316,0.487723,-1.304482,-0.476943,-0.478413,-0.419241,-0.286104,0.154209,-0.423562
3,-0.033455,0.069898,0.124634,0.625248,0.522717,0.607030,-1.311764,-0.391315,-0.404600,-0.385211,...,0.232208,-0.329589,-0.300537,0.345496,-0.476943,-0.478413,-0.419241,-0.286104,-0.685677,-0.423562
4,0.240241,0.171304,0.253336,-0.609620,-0.274470,-0.506961,0.611961,0.457681,0.476545,-0.385211,...,0.363316,0.988547,0.991947,1.350413,-0.476943,-0.473138,2.114952,-0.286104,1.156711,0.501534


In [143]:
X_test_scaled = pd.DataFrame(X_test_scaled, columns = X_train_rfe.columns)

In [144]:
X_test_scaled.head()

,arpu_6,arpu_7,arpu_8,onnet_mou_6,onnet_mou_7,onnet_mou_8,offnet_mou_6,offnet_mou_7,offnet_mou_8,roam_ic_mou_7,...,total_rech_amt_8,max_rech_amt_6,max_rech_amt_7,last_day_rch_amt_8,vol_2g_mb_7,vol_2g_mb_8,vol_3g_mb_8,monthly_2g_7,aon,aug_vbc_3g
0,-0.307306,-0.500317,-0.411171,0.076889,-0.124479,-0.036695,-0.098868,-0.437922,-0.354885,-0.385211,...,-0.315590,-0.015829,-0.300537,-1.304482,-0.476943,-0.478413,-0.419241,-0.286104,0.782817,-0.423562
1,0.686509,0.847940,0.609443,0.524741,0.234422,-0.112488,1.154660,1.327209,1.060259,-0.385211,...,0.695025,0.988547,0.991947,-1.304482,-0.476943,-0.478413,-0.419241,-0.286104,1.270792,-0.423562
2,0.535822,0.736384,1.042205,-0.633806,-0.580843,-0.526505,-0.847262,-0.230440,-0.207199,3.935578,...,1.027212,1.103036,1.419066,-1.304482,-0.476943,-0.478413,-0.419241,-0.286104,-1.403148,2.712611
3,-0.794948,0.127065,-0.976044,-0.793706,-1.119253,-1.691705,-0.342750,0.661367,-0.566649,-0.385211,...,-2.288544,0.474316,0.677906,-1.304482,-0.476943,-0.478413,-0.419241,-0.286104,0.410915,-0.423562
4,0.488021,0.691814,0.690242,1.463191,1.629232,1.716195,-0.091120,-0.611783,0.081492,-0.385211,...,0.737184,0.349105,0.590102,1.037975,-0.476943,-0.478413,-0.419241,-0.286104,-0.785938,-0.423562


In [167]:
#Saving the preprcessed data
X_test_scaled.to_csv('test_preprocessed.csv',index = False)

### Making predictions using Gradient boosting model

In [161]:
model_gb = joblib.load('GradientBoosting.pickle')

In [162]:
test_pred = model_gb.predict(X_test_scaled)

In [163]:
#Let'd create a submission file
df_test = pd.read_csv('test.csv')
results = df_test[['id']]

In [164]:
results['churn_probability'] = test_pred

In [165]:
results.head()

,id,churn_probability
0,69999,0
1,70000,0
2,70001,1
3,70002,0
4,70003,0


In [166]:
results.to_csv('Harshad_pred.csv',index = False)